In [1]:
from SpiderV2 import Spider
from IndexV2 import Database, NewIndex2, RawInfoIndex
from Tokenize import Tokenize
import time

In [2]:
db = Database(SearchEngine="Demo2")
index = NewIndex2()
raw_storage = RawInfoIndex()

In [7]:

url = "https://iot-kmutnb.github.io/"
#"https://en.wikipedia.org/wiki/Climate_change", "https://riskofrain.fandom.com/wiki/Main_Page", "https://www.thairath.co.th/news/local/bangkok/2635906" 
Spider.set_base_domains(["https://www.thairath.co.th/", "https://riskofrain.fandom.com/wiki/Main_Page", "https://en.wikipedia.org/", "https://iot-kmutnb.github.io/"])
Spider.max_depth = 0

spider = Spider(url)
success = spider.generate_next_soup()

raw_storage.read_from_database(db)

if success:
    info = spider.crawl({})
    raw_storage.modify_index(Spider.queue[Spider.queue_front-1], info[0], info[1], info[2])
    raw_storage.save_to_database(db)
else:
    if Spider.get_base_domain(url) in Spider.all_robots:
        Spider.unaccessible_urls.append(url)
    print("Cannot crawl to the website.")

Cannot crawl to the website.


In [8]:
print(Spider.unaccessible_urls)

['https://iot-kmutnb.github.io/', 'https://iot-kmutnb.github.io/']
